In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"



file_path = os.path.abspath(".")

execution_path = os.path.abspath(f"../../../../Binaries/Debug")

In [ ]:
import os

os.chdir(file_path)
import sys

target_path = os.path.abspath(f"../../../../Binaries/Debug")
sys.path.append(target_path)
print(f"Added {target_path} to sys.path")
package_path = os.path.abspath(f"../python")
sys.path.append(package_path)
print(f"Added {package_path} to sys.path")

In [3]:
import glints.scratch_grid
import glints.renderer
import glints.test_utils as test_utils
import torch
import numpy as np
import pytest

In [4]:
def linear_to_gamma(image):
    return image ** (1.0 / 2.2)


def gamma_to_linear(image):
    return image**2.2


import matplotlib.pyplot as plt


def save_images(field, resolution, divergence, smoothness):
    for i in range(field.field.shape[2]):
        test_utils.save_image(
            1000 * divergence[:, :, i], resolution, f"divergence_{i}.exr"
        )
        test_utils.save_image(
            100 * smoothness[:, :, i], resolution, f"smoothness_{i}.exr"
        )

        density = torch.norm(field.field[:, :, i], dim=2)
        directions = field.field[:, :, i] / density.unsqueeze(2)
        directions = torch.cat(
            [directions, torch.zeros_like(directions[:, :, :1])], dim=2
        )

        test_utils.save_image(directions, resolution, f"directions_{i}.exr")
        test_utils.save_image(density, resolution, f"density_{i}.exr")
        test_utils.save_image(field.field[:, :, i, :1], resolution, f"field_{i}.exr")

In [11]:
import importlib

importlib.reload(glints.scratch_grid)
os.chdir(execution_path)
r = glints.renderer.Renderer()
r.set_glints_roughness(0.0016 / 2)   # 划痕内部的粗糙程度，数值越大，划痕越模糊
vertices, indices = glints.renderer.plane_board_scene_vertices_and_indices()
camera_position_np = np.array([-2.0, 0.0, 6], dtype=np.float32) # 调参：相机位置，板子放在z=0平面上
r.set_look_at(
    camera_position_np,
    np.array([0.0, 0.0, 0.0], dtype=np.float32),
    np.array([-1.0, 0.0, 0.0], dtype=np.float32),
)
# r.set_camera_position(camera_position_np)
fov_in_degrees = 35 # 调参：视场角
resolution = [1200, 800] # 分辨率
r.set_perspective(
    np.pi * fov_in_degrees / 180.0, resolution[0] / resolution[1], 0.1, 1000.0
)
r.set_mesh(vertices, indices)
# r.set_light_position(torch.tensor([-2.0, 0.0, 5.5], device="cuda")) # 调参：光源位置
r.set_light_position(torch.tensor([-2.0, 0.0, 2.5], device="cuda")) # 调参：光源位置

r.set_width(torch.tensor([0.001], device="cuda"))

field = glints.scratch_grid.ScratchField(400, 4) # 调参：划痕场，分辨率为400，第二个参数是子场的个数，每个子场会多占一份显存，散度和光滑度只在一个子场上计算
image, sampled_mask = glints.scratch_grid.render_scratch_field(r, resolution, field)
test_utils.save_image(image, resolution, "scratch_field_initial.exr")
# target_image = r.prepare_target("texture.png", resolution)

loss_fn = (
    lambda x, y: torch.nn.functional.l1_loss(x, y) * 1
    + torch.nn.functional.mse_loss(x, y) * 1
) # 调参，损失函数组合，作用在图像上
regularization_loss_fn = torch.nn.HuberLoss() # 作用在散度、光滑度上，没啥可调的
regularizer = torch.optim.Adam([field.field], lr=0.0001) # 目前没用
optimizer = torch.optim.Adam([field.field], lr=0.0001)

In [12]:
target_count = 80 # 监督信号的数量
angle_range = 20 # 监督信号的角度范围

In [13]:
targets = []
import cv2

all_target_max = torch.tensor(0.0, device="cuda")

for i in range(target_count + 1):
    target = cv2.imread(f"targets/render_{i:03d}.exr", cv2.IMREAD_UNCHANGED)[..., :3]
    target = torch.tensor(target, dtype=torch.float32).cuda()
    target = torch.rot90(target, k=3, dims=[0, 1])
    targets.append(target)
    all_target_max = torch.max(target.max(), all_target_max)

In [14]:
# os.makedirs("feifan/scratch_field", exist_ok=True)
# os.makedirs("feifan/target", exist_ok=True)
# for sub_field_id in range(field.field.shape[2]):
#     os.makedirs(f"feifan/directions_{sub_field_id}", exist_ok=True)

In [ ]:
losses = []


for i in range(1000):

    rnd_pick_target_ids = np.random.randint(0, target_count + 1, size=4)
    init_id = np.random.randint(0, target_count + 1)
    rnd_pick_target_ids = np.arange(init_id, init_id + 4, 2) % (target_count + 1) # 调参， batch size
    # rnd_pick_target_ids = np.linspace(0, target_count, target_count + 1, dtype=np.int32)
    iterative_rnd_pick_target_id = i % (target_count + 1)
    # rnd_pick_target_ids[-1] = iterative_rnd_pick_target_id

    camera_rotate_angle = (
        iterative_rnd_pick_target_id * (angle_range / target_count) - angle_range / 2
    ) * (np.pi / 180)
    rotated_camera_position = test_utils.rotate_postion(
        camera_position_np,
        camera_rotate_angle,
        axis=np.array([-1, 0, 0], dtype=np.float32),
    )
    r.set_look_at(
        rotated_camera_position,
        np.array([0.0, 0.0, 0.0], dtype=np.float32),
        np.array([-1.0, 0.0, 0.0], dtype=np.float32),
    )
    image, sampled_mask = glints.scratch_grid.render_scratch_field(r, resolution, field)
    test_utils.save_image(image, resolution, f"feifan/scratch_field/scratch_field_{i:03}.exr")
    test_utils.save_image(
        targets[iterative_rnd_pick_target_id], resolution, f"feifan/target/target_{i:03}.exr"
    )

    field.fix_direction()

    for sub_field_id in range(field.field.shape[2]):
        directions = torch.rot90(field.field[:, :, sub_field_id, :2])
        test_utils.plot_arrows(
            directions,
            "directions",
            spacing=6,
            filename=f"feifan/directions_{sub_field_id}/directions_{sub_field_id}_{i:03}.png",
        )
        plt.close()

    torch.cuda.empty_cache()

    camera_positions = []
    target_images = []

    for rnd_pick_target_id in rnd_pick_target_ids:

        camera_rotate_angle = (
            rnd_pick_target_id * (angle_range / target_count) - angle_range / 2
        ) * (np.pi / 180)
        rotated_camera_position = test_utils.rotate_postion(
            camera_position_np,
            camera_rotate_angle,
            axis=np.array([-1, 0, 0], dtype=np.float32),
        )
        camera_positions.append(rotated_camera_position)
        target_images.append((targets[rnd_pick_target_id] / all_target_max) * 30)

    print(f"iteration {i}, target {rnd_pick_target_id}")

    losses += glints.scratch_grid.optimize_field(
        field,
        r,
        resolution,
        target_images,
        loss_fn,
        regularization_loss_fn,
        regularizer,
        optimizer,
        epochs=1,
        enable_smoothness_regularization=True, # 调参，是否启用光滑度正则化
        enable_divergence_regularization=True, # 调参，是否启用散度正则化
        camera_positions=camera_positions,
    )


plt.plot(losses)
plt.show()

In [ ]:
camera_rotate_angle = (10 * (20.0 / 20) - 10) * (np.pi / 180)
rotated_camera_position = test_utils.rotate_postion(
    camera_position_np,
    camera_rotate_angle,
    axis=np.array([-1, 0, 0], dtype=np.float32),
)
r.set_look_at(
    rotated_camera_position,
    np.array([0.0, 0.0, 0.0], dtype=np.float32),
    np.array([-1.0, 0.0, 0.0], dtype=np.float32),
)
image, sampled_mask = glints.scratch_grid.render_scratch_field(r, resolution, field)
test_utils.save_image(image, resolution, "scratch_field.exr")

# test_utils.save_image(gamma_to_linear(target_image), resolution, "target_image.exr")
divergence, smoothness = field.calc_divergence_smoothness()
save_images(field, resolution, divergence, smoothness)

image, sampled_mask = glints.scratch_grid.render_scratch_field(r, resolution, field)
test_utils.save_image(image, resolution, "scratch_field.exr")

directions = torch.rot90(field.field[:, :, 0, :2])


test_utils.plot_arrows(directions, "directions", spacing=6, filename="directions.pdf")

In [10]:
# field.field.requires_grad = False
# field.field = field.field.detach()
# torch.cuda.empty_cache()

In [11]:
# line_counts = []
# for density_held in np.linspace(2.0, 3.0, 9):
#     lines = field.discretize_to_lines(density_held, 0.4)
#     line_counts.append(lines.shape[0])


# plt.plot(np.linspace(2.0, 3.0, 9), line_counts)
# plt.xlabel("Density held")
# plt.ylabel("Line count")
# plt.savefig("line_count.pdf")
# plt.show()

In [ ]:
lines = field.discretize_to_lines(
    0.5, 0.5
)  # 划痕离散化，卡住了调，第一个每根划痕带走的density，可以超过1，一般越大带走越快，每根越短，第二个参数是结束的阈值，不能超过1，越接近1结束越快
print(lines.shape)
lines = lines.cpu()


In [ ]:
torch.save(lines, "lines.pt")
a = torch.load("lines.pt")
print(a.shape)

In [13]:
# torch.cuda.empty_cache()
r.set_type("bspline")


for i in range(target_count + 1):
    camera_rotate_angle = (i * (angle_range / target_count) - angle_range / 2) * (
        np.pi / 180
    )
    rotated_camera_position = test_utils.rotate_postion(
        camera_position_np,
        camera_rotate_angle,
        axis=np.array([-1, 0, 0], dtype=np.float32),
    )
    r.set_look_at(
        rotated_camera_position,
        np.array([0.0, 0.0, 0.0], dtype=np.float32),
        np.array([-1.0, 0.0, 0.0], dtype=np.float32),
    )

    image, _ = r.render(resolution, lines)
    test_utils.save_image(image, resolution, f"bspline_{i:03}.exr")

In [14]:
# os.chdir(execution_path)
# import torch
# lines = torch.load("lines.pt")

# r = glints.renderer.Renderer()

# vertices, indices = glints.renderer.plane_board_scene_vertices_and_indices()
# camera_position_np = np.array([4.0, 0.0, 3.5], dtype=np.float32)
# r.set_camera_position(camera_position_np)
# fov_in_degrees = 35
# resolution = [768 * 2, 512 * 2]
# r.set_perspective(
#     np.pi * fov_in_degrees / 180.0, resolution[0] / resolution[1], 0.1, 1000.0
# )
# r.set_mesh(vertices, indices)
# r.set_light_position(torch.tensor([0.0, 4.0, 4.5], device="cuda"))

# r.set_width(torch.tensor([0.001], device="cuda"))